#### Copyright 2019 Google LLC.

In [0]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Embeddings

Embeddings are a powerful way to represent data for deep learning (neural network) models.

In this exercise, we will work specifically with embeddings of words. This is one of the most common applications of this technique. We will train word embeddings from scratch on a small dataset and visualize those embeddings.

---

<img height="20px" src="https://upload.wikimedia.org/wikipedia/commons/2/2d/Tensorflow_logo.svg" align="left" hspace="10px">
This lab is developed based on [TensorFlow.org's Word embeddings](https://www.tensorflow.org/alpha/tutorials/sequences/word_embeddings).

# Learn embeddings



Embeddings can be trained using either supervised or unsupervised learning. For this exercise, we will train a supervised sentiment classifier on IMDB movie reviews. In the process, we will learn our own word embeddings.

### Setup

First, we will set random seeds for reproducible results. This makes sure that we get the same results every time we run this lab, by controlling how any random numbers are generated. **This should almost never be used in production code.**

In [0]:
import numpy as np
import tensorflow as tf

np.random.seed(42)
tf.set_random_seed(42)

## Data and Problem

We will use the [IMDB dataset](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb) that contains the text of 50,000 movie reviews from the [Internet Movie Database](https://www.imdb.com/). This dataset is conveniently packaged in Keras. 

In [0]:
from tensorflow import keras

imdb = keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(
    index_from=2)

The reviews have been pre-processed so that the text of each review (a list of words) has been converted to a list of integers, where each integer represents a specific word. We set the smallest word index as 3 (`index_from=2`) to handle some special tokens, which we'll describe in the next section.

Each sentiment label is an integer value of either 0 or 1, where 0 is a negative review, and 1 is a positive review. Thus this is a binary classification problem.

In [0]:
print(train_data[0])

### Convert the integers back to words

It may be useful to know how to convert a list of integers back to text. Here, we'll create a helper function to query a dictionary object that contains the integer to string mapping:

In [0]:
# A dictionary mapping each word to an integer index.
word_index = imdb.get_word_index()

# Reserve the first 3 indices for special tokens.
word_index = {k:(v+2) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2

vocab_size = len(word_index)
print('# unique words: {}'.format(vocab_size))

In [0]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

decode_review(train_data[0])

### Preprocessing

Note that there are three special tokens in the `word_index` dictionary:

- `"<START>"` marks the beginning of each review
- `"<UNK>"` handles "unknown" words, or words that appear in the test dataset but not in the training dataset (so they are effectively "unknown" to the model)
- `"<PAD>"` adds extra tokens to the end of shorter reviews

Movie reviews can be different lengths, but inputs to a neural network must all be the same length. Let's take a look at the distribution of review lengths.

In [0]:
import matplotlib.pyplot as plt

review_lengths = [len(review) for review in train_data]
plt.hist(review_lengths, density=True, cumulative=True)
# plt.yticks(np.arange(0, 1.1, step=0.1))
plt.show()

Even though the longest review is close to 2500 words long, over 90% of the reviews have fewer than 500 words. 

We can use the `pad_sequences` function to standardize the lengths of the reviews to 500 words long. Any reviews longer than this will have the extra words truncated, while any reviews shorter than 500 words will have extra `"<PAD>"` tokens added to the end. 

Choosing a standardized length requires balancing efficiency (longer lengths mean slower training) and information loss (shorter lengths may truncate too much valuable information). Aiming for a length that fully covers 90% of samples is generally reasonable, and you can further tune this as a hyperparameter.

In [0]:
maxlen = 500

train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=maxlen)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=maxlen)

Let's inspect the first padded review.

In [0]:
print(train_data[0])

## Build the Model

### Using the Embedding layer

Keras makes it easy to use embeddings. The [Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) layer takes at least two arguments: the number of possible words in the vocabulary and the dimensionality of the embeddings. We will start by using a small embedding size of 2 to make visualization easier.

In [0]:
embedding_dim = 2

embedding_layer = keras.layers.Embedding(vocab_size, embedding_dim)

The Embedding layer can be understood as a lookup table that maps from integer indices (which stand for specific words) to dense vectors (their embeddings). The dimensionality (or width) of the embedding is a parameter you can experiment with to see what works well for your problem, much in the same way you would experiment with the number of neurons in a Dense layer.

When we create an Embedding layer, the weights for the embedding are randomly initialized (just like any other layer). During training, they are gradually adjusted via backpropagation. Once trained, the learned word embeddings will roughly encode similarities between words (as they were learned for the specific problem our model is trained on).

As input, the Embedding layer takes a 2D tensor of integers, of shape `(num_samples, sequence_length)`, where each sample is a sequence of integers. As output, the embedding layer returns a 3D floating point tensor, of shape `(num_samples, sequence_length, embedding_dimensionality)`. 

### Configure the Model

1. The first layer is an Embedding layer. This layer takes the integer-encoded vocabulary and looks up the embedding vector for each word-index. These vectors are learned as the model trains. The vectors add a dimension to the output array. The resulting dimensions are: `(batch size, sequence length, embedding size)`.

1. Next, we flatten the output from a 2-d array to a 1-d array.

1. The last layer is densely connected with a single output node. Using the sigmoid activation function, this value is a float between 0 and 1, representing a probability (or confidence level) that the review is positive.

In [0]:
model = keras.Sequential([
  keras.layers.Embedding(vocab_size, embedding_dim, input_length=maxlen),
  keras.layers.Flatten(),
  keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

### Compile and train the model

In [0]:
model.compile(
  # Calculate loss for a binary classification problem.
  loss='binary_crossentropy',

  # Adam is one of the most commonly used optimizers.
  optimizer=tf.train.AdamOptimizer(),

  # We will only track accuracy for this task.
  metrics=['accuracy']
)

The number of epochs and batch size used during training are both hyperparameters, meaning you may need to experiment with different values to improve performance. This also means there's no magic answer for which values to choose.

If you've implemented your model correctly, you should observe decreasing training loss within a few epochs. With more epochs (more training), the model will experience more overfitting. You will typically train for at least 5 epochs; for this model, we train for 10 epochs as a reasonable tradeoff between learning (as the validation accuracy is still increasing) and overfitting.

Since we have a fairly large dataset, we also want to process data in batches instead of the entire dataset at once. If we choose batch sizes that are too small, we will get slower training per epoch, while larger batch sizes may require more epochs to train and even cause out-of-memory errors. It's good practice to start with a small power of two (e.g. 32), then experiment with continuously doubling your batch size. For this model, we are able to use a batch size of 512 with good performance.

In [0]:
# Train the model.
history = model.fit(
    train_data,
    train_labels,
    epochs=10,
    batch_size=512,
    validation_split=0.2
)

In [0]:
acc = history.history['acc']
val_acc = history.history['val_acc']

epochs = range(1, len(acc) + 1)

plt.figure(figsize=(12,9))
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim((0.5,1))

plt.show()

With this approach our model reaches a validation accuracy of around 87%. (Note that the model is already beginning to overfit, as reflected in the diverging training and validation accuracy curves.)

# Visualize embeddings

## Retrieve the learned embeddings

Next, let's retrieve the word embeddings learned during training. This will be a matrix of shape `(vocab size, embedding dimension)`.

In [0]:
e = model.layers[0]
embedding_matrix = e.get_weights()[0]
print(embedding_matrix.shape)

Let's explore the learned word embeddings on a small set of terms (looking at all 80,000+ at once would be overwhelming). Most of these terms are generally strongly indicative of sentiment, but some are added just for fun.

In [0]:
informative_terms = [ "bad", "great", "best", "worst", "fun", "beautiful",
                      "excellent", "poor", "boring", "awful", "terrible",
                      "definitely", "perfect", "liked", "worse", "waste",
                      "entertaining", "loved", "unfortunately", "amazing",
                      "enjoyed", "favorite", "horrible", "brilliant", "highly",
                      "simple", "annoying", "today", "hilarious", "enjoyable",
                      "dull", "fantastic", "poorly", "fails", "disappointing",
                      "disappointment", "not", "him", "her", "good", "time",
                      "sad", "exciting", "slow", "movie", "film", "action",
                      "comedy", "drama", "fabulous"]

Now we plot each word in `informative_terms` on a 2-dimensional graph. Recall that we are using 2-dimensional embeddings, so consider the 1st value as an x-value and the 2nd value as a y-value. 

*Note: If the displayed plot is too small, try running the cell again.*

In [0]:
for word in informative_terms:
  word_num = word_index[word]
  embeddings = embedding_matrix[word_num]
  plt.text(embeddings[0], embeddings[1], word)

# Do a little set-up to make sure the plot displays nicely.
plt.rcParams["figure.figsize"] = (25, 25)
plt.xlim(1.2 * embedding_matrix.min(), 1.2 * embedding_matrix.max())
plt.ylim(1.2 * embedding_matrix.min(), 1.2 * embedding_matrix.max())
plt.show()

# Exercises

## Exercise 1

What structures do you see in the embeddings of the words?

### Student Solution

**Your answer here: **



### Answer Key

**Solution**

In [0]:
# TODO

**Validation**

In [0]:
# TODO

## Exercise 2: Higher-dimensional embeddings

Now we can visualize 2-dimensional embeddings, but recall that our original vocabulary had over 80,000 words; we're missing a lot of information if we only use 2 dimensions!

Typical embedding dimensions are 50, 100, 200, 300, and sometimes even larger. In this section we'll introduce the [Embedding Projector](http://projector.tensorflow.org/), a tool to visualize high-dimensional embeddings.

Re-train our earlier model, with the same hyperparameters, using **50-dimensional** embeddings and retrieve the learned embeddings from this trained model. You should observe a slower training time and much higher training accuracy after 10 epochs.

### Student Solution

In [0]:
### YOUR CODE HERE ###

# Re-train model using 50-dimensional embeddings.

embedding_matrix = ...

### Answer Key

**Solution**

In [0]:
# Re-train model using 50-dimensional embeddings.

embedding_dim = 50

# Keep the same model and training structure.

model = keras.Sequential([
  keras.layers.Embedding(vocab_size, embedding_dim, input_length=maxlen),
  keras.layers.Flatten(),
  keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
  loss='binary_crossentropy',
  optimizer=tf.train.AdamOptimizer(),
  metrics=['accuracy']
)

history = model.fit(
    train_data,
    train_labels,
    epochs=10,
    batch_size=512,
    validation_split=0.2
)

# Retrieve embeddings.

e = model.layers[0]
embedding_matrix = e.get_weights()[0]
print(embedding_matrix.shape)

**Validation**

In [0]:
# TODO

We will now write the embeddings to disk. To use the Embedding Projector, we will upload two files in tab separated format: a file of vectors (containing the embeddings), and a file of metadata (containing the words). We will again only analyze our small set of terms from above.

In [0]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word in informative_terms:
  word_num = word_index[word]
  embeddings = embedding_matrix[word_num]
  # for compatibility with python 2 and 3
  out_m.write(word.encode("utf-8").decode("utf-8")  + "\n")
  out_v.write('\t'.join(
      [str(x).encode("utf-8").decode("utf-8") for x in embeddings]) + "\n")
out_v.close()
out_m.close()

try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

Finally, open the [Embedding Projector](http://projector.tensorflow.org/):

*   Click on "Load data".
*   Upload the two files we created above: `vecs.tsv` and `meta.tsv`.

The embeddings we have trained will now be displayed. You can search for words to find their closest neighbors.

*Note: Your results may be a bit different, depending on how weights were randomly initialized before training the embedding layer.*

## Exercise 3

How do the structures of these embeddings compare to the 2-dimensional embeddings we visualized earlier? How do the model accuracies compare? What might this suggest about the effect of increasing embedding size?

### Student Solution

**Your answer here:**

### Answer Key

**Solution**

In [0]:
# TODO

**Validation**

In [0]:
# TODO

## Exercise 4: Beyond word embeddings

The Embedding layer in Keras makes it easy to represent any data that has an index-based lookup. What are some other kinds of data (besides words) that this could be useful for?

On the flip side, what are some kinds of data that don't work as well with an index-based lookup? Let's say we still want to achieve the advantages of using embeddings, namely transforming this data into 1) efficient, dense vectors that 2) capture similarities between data. How could we do this?

### Student Solution

**Your answer here:**

### Answer Key

**Solution**